# Misura del momento d'inerzia di un pendolo composto
## Setup dell'ambiente di lavoro

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as numpy
from scipy.optimize import curve_fit
import math

matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'text.usetex': True,
    'pgf.rcfonts': False,
})

## Importazione dei dati
Importiamo i dati da un file CSV ottenuto a partire da quello fornito.

In [2]:
pendolo_df = pd.read_csv('Dati_pronti.csv', sep=';', decimal=',')

distanza_d = pendolo_df['distanza_d'].dropna()
periodo = pendolo_df['periodo'].dropna()
distanza_r = pendolo_df['distanza_r'].dropna()

massa_pendolo = pendolo_df['massa_pendolo'][0]
lunghezza_sbarra= pendolo_df['lunghezza_asta'][0]
massa_mobile = pendolo_df['massa_mobile'][0]
massa_fissa = pendolo_df['massa_fissa'][0]
df = pendolo_df['df'][0]
ds = pendolo_df['ds'][0]
raggio_dischi = pendolo_df['raggio_dischi'][0]

## Metodo 1
### Calcolo 

In [3]:
def calcola_momento_inerzia(M, g, r, T):
    return (M * g * r * T ** 2) / (4 * math.pi ** 2)

momento_inerzia = calcola_momento_inerzia(massa_pendolo, 9.81, distanza_r, periodo)

### Errore

In [4]:
def calcola_sigma2_i(g, T, r, M, sigma_m, sigma_t, sigma_r):
    return ((g / (4 * (math.pi ** 2))) ** 2) * ((T ** 4) * (r ** 2) * (sigma_m ** 2) +
                                            4 * (M ** 2) * (T ** 2) * (r ** 2) * (sigma_t ** 2) +
                                            (M ** 2) * (T ** 4) * (sigma_r ** 2))

sigma_i = numpy.sqrt(calcola_sigma2_i(9.81, periodo, distanza_r, massa_pendolo, 0.001, 0.01, 0.001))

### Grafico

In [5]:
func_x = numpy.linspace(0.1, 0.9, 1000)

def parabola(x, a, b, c):
    return a * (x ** 2) + b * x + c

fitted_params, var_matrix = curve_fit(parabola, distanza_d, momento_inerzia)

func_y = parabola(func_x, fitted_params[0], fitted_params[1], fitted_params[2])

plt.plot(func_x, func_y, 'r--', zorder=2, label='Fit quadratico')
plt.errorbar(distanza_d, momento_inerzia, yerr=sigma_i,
            ecolor='xkcd:black', capsize=5, fmt='k', zorder=1, label="Momento d'inerzia (metodo 1)",
            ls='none')

plt.legend(loc='upper left', shadow=True)                
plt.xlabel(r"Distanza $d$ ($m$)")
plt.ylabel(r"Momento d'inerzia $I$ ($kg \cdot m^2$)")
plt.title(r"Momento d'inerzia del pendolo al variare della distanza $d$")

plt.savefig('metodo1.pgf')

plt.show()
plt.close()


## Metodo 2
### Calcolo

In [6]:
def calcola_momento_inerzia2(m_s, L, d_s, m_d, r_d, d_f, d):
    return (1/12) * m_s * (L ** 2) + m_s * (d_s ** 2) + m_d * (r_d ** 2) + m_d * (d_f ** 2) + m_d * (d              ** 2)

massa_sbarra = massa_pendolo - (massa_fissa + massa_mobile)

momento_inerzia = calcola_momento_inerzia2(massa_sbarra, lunghezza_sbarra, ds, massa_mobile, raggio_dischi, df, distanza_d)

### Grafico

In [7]:
func_x = numpy.linspace(0.1, 0.9, 1000)

plt.plot(func_x,
         calcola_momento_inerzia2(massa_sbarra, lunghezza_sbarra, ds, massa_mobile, raggio_dischi, df,          func_x), label="Momento d'inerzia (metodo 2)")

plt.legend(loc='upper left', shadow=True)                
plt.xlabel(r"Distanza $d$ ($m$)")
plt.ylabel(r"Momento d'inerzia $I$ ($kg \cdot m^2$)")
plt.title(r"Momento d'inerzia del pendolo al variare della distanza $d$ (confronto)")

plt.savefig('metodi12.pgf')

plt.show()
plt.close()